**ELMo**: contextual embedding


In [1]:
!python -m spacy download en_core_web_md

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
!pip install tensorflow==1.15

In [21]:
!pip install chart_studio

     |████████████████████████████████| 71kB 2.1MB/s 


In [5]:
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing

import spacy
from spacy.lang.en import English
from spacy import displacy
nlp = spacy.load('en_core_web_md')
from IPython.display import HTML
import logging
logging.getLogger('tensorflow').disabled = True #OPTIONAL - to disable outputs from Tensorflow

1. Load Data

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
input_path = "/content/drive/My Drive/colabdata/input"

In [8]:
train = pd.read_csv(f'{input_path}/train.csv')

In [9]:
train.head()

,company,country,sector,subsector,summaries,info_from_official_confidence,vague_products,products,services,valuechains,industries,company profile,industry_3,activities_3,sector_3,sub_industry_3,company_type_3,company_profile_3
0,Vilondo,indonesia,TMT,digital_business,vilondo is a booking engine for holiday villa ...,1.000000,book hotel book book book book book restaurant...,beach beach adventure sport,NaN,NaN,NaN,the villas are ideal for families and so is th...,NaN,NaN,NaN,NaN,NaN,NaN
1,ANN SIANG PROPERTY HOLDINGS PTE LTD,NaN,CNI_REH,NaN,ann siang property holdings pte ltd is a compa...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FOXCONN INDUSTRIAL INTERNET CO LTD,china,TMT,semiconductor,mar 31 2019 foxconn industrial internet is a s...,1.000000,network network equipment cloud platform netwo...,electronic equipment product electronic equipm...,communication network equipment cloud service ...,design manufacturing provider manufacturing de...,NaN,industrial internet solutions can achieve syst...,communications equipment,NaN,NaN,NaN,other communications equipment manufacturing,foxconn industrial internet co ltd is a china ...
3,"CHINA FILM CO., LTD.",china,TMT,media,jan 15 2020 company profile page for china fil...,0.869079,NaN,NaN,NaN,NaN,NaN,NaN,motion pictures media,NaN,communications,entertainment content,NaN,china film co ltd is a china based company pri...
4,L.S. CONSTRUCTION PTE LTD,NaN,CNI_REH,buildings_&_industrial,ls construction engineering pte ltd bizsafe le...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,specialty trade contractors personal and laund...,NaN,NaN,NaN,l s construction pte ltd is a company in singa...


2. Prepare sentences

In [30]:
train = train.fillna('')
texts = list(train['company_profile_3'])[0:100]

text = ' '.join(texts)

In [31]:
len(text)

35030

In [32]:
import re

sentences = []

text = re.sub('\s+', ' ', text.lower())
doc = nlp(text)
for i in doc.sents:
  if len(i)>1:
    sentences.append(i.string.strip())
      
len(sentences)

189

In [33]:
sentences[0:10]

['foxconn industrial internet co ltd is a china based company principally engaged in the research development manufacture and sales of electronic equipment the company s main products include communications network equipment cloud service equipment precision tools and industrial robots the company s products are applied in smart phones broadband and wireless networks',
 'the infrastructure of multimedia service operators telecom operators and enterprise networks and data centers as well as terminal products needed by internet value',
 'added service providers and the automatic intelligent manufacturing of precision core components the company distributes its products in domestic market and to overseas markets',
 'china film co ltd is a china based company principally engaged in the investment design production and release of films and the operation of movie theatres',
 'the company mainly operates four segments including production of film and television release of movies film projecti

3. Prepare sentence embedding

In [34]:
import tensorflow as tf
print(tf.__version__)

import tensorflow_hub as hub

1.15.0


In [35]:
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

In [36]:
embeddings = embed(
    sentences,
    signature="default",
    as_dict=True)["default"]

In [37]:
%%time
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(tf.tables_initializer())
  x = sess.run(embeddings)

CPU times: user 4 s, sys: 968 ms, total: 4.97 s
Wall time: 4.62 s


## Visualize the sentences using PCA and TSNE

In [38]:
from sklearn.decomposition import PCA

pca = PCA(n_components=20)
y = pca.fit_transform(x)

from sklearn.manifold import TSNE

y = TSNE(n_components=2).fit_transform(y)

In [39]:
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)


data = [
    go.Scatter(
        x=[i[0] for i in y],
        y=[i[1] for i in y],
        mode='markers',
        text=[i for i in sentences],
    marker=dict(
        size=16,
        color = [len(i) for i in sentences], #set color equal to a variable
        opacity= 0.8,
        colorscale='Viridis',
        showscale=False
    )
    )
]
layout = go.Layout()
layout = dict(
              yaxis = dict(zeroline = False),
              xaxis = dict(zeroline = False)
             )
fig = go.Figure(data=data, layout=layout)
file = plot(fig, filename='Sentence encode.html')

from google.colab import files
files.download('Sentence encode.html') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [43]:
#@title Sementic search
#@markdown Enter a set of words to find matching sentences. 'results_returned' can beused to modify the number of matching sentences retured. To view the code behind this cell, use the menu in the top right to unhide...
search_string = "electricity" #@param {type:"string"}
results_returned = "3" #@param [1, 2, 3]

from sklearn.metrics.pairwise import cosine_similarity


embeddings2 = embed(
    [search_string],
    signature="default",
    as_dict=True)["default"]

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(tf.tables_initializer())
  search_vect = sess.run(embeddings2)
  

cosine_similarities = pd.Series(cosine_similarity(search_vect, x).flatten())
output =""
for i,j in cosine_similarities.nlargest(int(results_returned)).iteritems():
  output +='<p style="font-family:verdana; font-size:110%;"> '
  for i in sentences[i].split():
    if i.lower() in search_string:
      output += " <b>"+str(i)+"</b>"
    else:
      output += " "+str(i)
  output += "</p><hr>"
    
output = '<h3>Results:</h3>'+output
display(HTML(output))
#   print(sentences[i])
#   print('\n')